In [6]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [7]:
# %cd drive/MyDrive/collab_sandbox/persona_bot_sirius/

In [8]:
# ! pip -q install transformers

## Model initial configuration

In [9]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
import json
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [10]:
SPECIAL_TOKENS = [
	 '<speaker_1>', 
	 '</speaker_1>', 
	 
	 '<speaker_2>',
	 '</speaker_2>',

	 '<persona>',
	 '</persona>'
]
SPECIAL_TOKENS = {item:item for item in SPECIAL_TOKENS}

## Persona chat dataset

In [11]:
persona_chat_original = pd.read_csv("./persona_chat.csv")
persona_chat_original = persona_chat_original[:10]
persona_chat_original.head(3)

,Unnamed: 0,Persona,chat
0,0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."
1,1,my mom is my best friend. i have four sisters...,"hi , how are you doing today ?\ni am spending ..."
2,2,i had a gig at local theater last night. i wo...,"we all live in a yellow submarine , a yellow s..."


In [12]:
class PersonaChatGenerator:
	def __init__(self, 
		initial_dataset=None,
	):
		self.initial_dataset = initial_dataset
		self.processed_dataset = []
		self.process_dataset()

	def process_dataset(self):
		processed_dataset = {
			"persona": [],
			"history": [],
			# "target": []
		}

		speaker_1_start = SPECIAL_TOKENS['<speaker_1>']
		speaker_1_end = SPECIAL_TOKENS['</speaker_1>']
		
		speaker_2_start = SPECIAL_TOKENS['<speaker_2>']
		speaker_2_end = SPECIAL_TOKENS['</speaker_2>']

		for i in range(len(self.initial_dataset)):
			persona = self.initial_dataset['Persona'].iloc[i]
			persona = f"{SPECIAL_TOKENS['<persona>']} {persona} {SPECIAL_TOKENS['</persona>']}"
			chat = self.initial_dataset['chat'].iloc[i].split("\n")
			chat = chat[:-1]
			history = ""
			for j in range(len(chat)):
				reply = chat[j]
				if (j+1) % 2 == 0:
					reply = f"{speaker_2_start} {reply} {speaker_2_end}"
					history += reply

					processed_dataset['persona'].append(persona)
					processed_dataset['history'].append(history)
					# processed_dataset['target'].append(reply)

				else:
					reply = f"{speaker_1_start} {reply} {speaker_1_end}"
					history += reply 

		dataset = pd.DataFrame(data=processed_dataset)
		return dataset

train_dataset_csv, valid_dataset_csv = train_test_split(persona_chat_original, test_size=0.1)

train_dataset_generator = PersonaChatGenerator(
	initial_dataset=train_dataset_csv,
)

valid_dataset_generator = PersonaChatGenerator(
	initial_dataset=valid_dataset_csv,
)



In [13]:
class PersonaChatDataset(Dataset):
	def __init__(self, 
		initial_dataset=None,
		tokenizer=None
	):
		self.initial_dataset = initial_dataset
		self.tokenizer = tokenizer
	
	def __len__(self):
		return len(self.initial_dataset)
	
	def __getitem__(self, idx):
		row = self.initial_dataset.iloc[idx]
		persona = [item.strip() for item in row['persona'].split(".") if len(item) > 0 ]
		random.shuffle(persona)
		persona = [torch.tensor(self.tokenizer.encode(item)).flatten() for item in persona]
		persona = torch.cat([*persona])

		history = row['history']
		history = self.tokenizer.encode(history)
		history = torch.tensor(history).flatten()
		
		# target = row['target']
		# target = torch.tensor(self.tokenizer.encode(target)).flatten()
		# target = torch.cat([target, torch.tensor([self.tokenizer.eos_token_id], dtype=torch.long)])

		# feature = torch.cat([persona, history, torch.tensor([tokenizer.eos_token_id])])
		feature = torch.cat([persona, history])

		# target = torch.tensor(target, dtype=torch.long)
		# target = target.flatten()
		return {
			"feature": feature,
			"target": feature 
		}


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
tokenizer.add_tokens(list(SPECIAL_TOKENS.values()), special_tokens=True)

train_dataset = PersonaChatDataset(
	initial_dataset=train_dataset_generator.process_dataset(),
	tokenizer=tokenizer
)

valid_dataset = PersonaChatDataset(
	initial_dataset=valid_dataset_generator.process_dataset(),
	tokenizer=tokenizer
)

In [14]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small-test'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 2
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [15]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return valid_dataset if evaluate else train_dataset


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

## Training and Evaluating

There will be quite a lot of code needed for training our model but don’t worry, everything should work as is, the main thing is to give the model the dataset in the right format.

Image from [Giphy](https://giphy.com/)

In [16]:
def train(args: Args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size

    def collate(examples):
        # print("EXAMPLES", examples)
        features = [item['feature'] for item in examples]
        features = pad_sequence(features, batch_first=True)
        
        targets = [item['target'] for item in examples]
        targets = pad_sequence(targets, batch_first=True)

        return {
            "feature": torch.tensor(features, dtype=torch.long),
            "target": torch.tensor(targets, dtype=torch.long)
        }

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model.resize_token_embeddings(len(tokenizer))

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    # Train!
    global_step = 1
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch['feature'], batch['target'])
            # print(inputs.shape, labels.shape, inputs, labels)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

        
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            
            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if args.evaluate_during_training:
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))

                    # _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        features = [item['feature'] for item in examples]
        features = pad_sequence(features, batch_first=True)
        
        targets = [item['target'] for item in examples]
        targets = pad_sequence(targets, batch_first=True)

        return {
            "feature": torch.tensor(features, dtype=torch.long),
            "target": torch.tensor(targets, dtype=torch.long)
        }

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch['feature'], batch['target'])
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        for key in sorted(result.keys()):
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [17]:
# Main runner

def main(df_trn, df_val):
    args = Args()

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    tokenizer.add_tokens(list(SPECIAL_TOKENS.values()), special_tokens=True)
    
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.resize_token_embeddings(len(tokenizer))
    model.to(args.device)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    # ...
    results = {}

    return results

It is time to train our model!

In [18]:
main(train_dataset, valid_dataset)

/home/dimweb/.local/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:969: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/home/dimweb/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

/tmp/ipykernel_22388/1749540694.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "feature": torch.tensor(features, dtype=torch.long),
/tmp/ipykernel_22388/1749540694.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "target": torch.tensor(targets, dtype=torch.long)


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 10.92 GiB total capacity; 1.34 GiB already allocated; 6.31 MiB free; 1.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
valid_dataset_csv = valid_dataset_csv.reset_index()
valid_dataset_csv['Persona'][0]

' i like to smell my own farts. my beer gut is so huge i haven t seen my feet in two years. i am from san fransico. i am always the one who buys the beers. i like to place blame on other people even when i know it is my fault.'

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

## Chatting with  Bot

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
tokenizer.add_tokens(list(SPECIAL_TOKENS.values()), special_tokens=True)
model = AutoModelWithLMHead.from_pretrained('output-small')

In [ ]:
# valid_dataset_csv = valid_dataset_csv.reset_index()
persona = valid_dataset_csv.iloc[6]['Persona']
print(persona)
persona = f"{SPECIAL_TOKENS['<persona>']} {persona} {SPECIAL_TOKENS['</persona>']}"
chat_history_ids = tokenizer.encode(persona, return_tensors='pt')
VOCAB_TOKENS = tokenizer.get_added_vocab()

user_inputs = [
    "Hi. What is your name?",
    "What do you like?",
    "What is your job?"
]

last_index = 0

for step in range(len(user_inputs)):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    # input('>> User:')
    print(user_inputs[step])
    user_input = f"{SPECIAL_TOKENS['<speaker_1>']} {user_inputs[step]} {SPECIAL_TOKENS['</speaker_1>']}{SPECIAL_TOKENS['<speaker_2>']}"
    user_input
    new_user_input_ids = tokenizer.encode(user_input, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

    # generated a response while limiting the total chat history to 1000 tokens, 
    model_response = model.generate(
        bot_input_ids, max_length=250,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100,
        top_p=0.7,
        temperature = 0.8,
    )

    # print("-"*100)
    # print(tokenizer.decode(model_response[0]))
    # print("-"*100)
    # print(list(model_response[0]), list(model_response[0]).index(VOCAB_TOKENS["</speaker_2>"]))
    model_response_list = list(model_response[0])
    end_speaker_index = last_index+model_response_list[last_index+1:].index(VOCAB_TOKENS["</speaker_2>"]) +1
    last_index = end_speaker_index
    model_response = model_response[:, :end_speaker_index+1]
    # print(chat_history_ids, torch.tensor([[VOCAB_TOKENS['<speaker_2>']]], dtype=torch.long), model_response)

    chat_history_ids = model_response
    # print("-"*100)
    # print(tokenizer.decode(chat_history_ids[0]))
    # print("-"*100)
    # chat_history_ids = torch.cat([
    #     chat_history_ids, 
    #     torch.tensor([[VOCAB_TOKENS['</speaker_2>']]], dtype=torch.long), 
    # ], dim=1)
    # print(model_response)
    # pretty print last ouput tokens from bot
    # print(chat_history_ids)
    print(f"Bot: {tokenizer.decode(chat_history_ids[0][len(bot_input_ids[0])-1:], skip_special_tokens=True)}")

 my favorite food is spaghetti and meatballs. i was raised by two mothers. i am not afraid of what others think. my boyfriend works for nasa. i can be quite forgetful.
Hi. What is your name?
Bot:  i am mark. nice to meet you 
What do you like?
Bot:  pizza. i am a huge fan of spaghetti and italian meatballs 
What is your job?
Bot:  my job is in the military. i have a degree in biology 
